# Jonathan Halverson
# Keeping it Fresh: Predict Restaurant Inspections
## Part 7: Yelp user reviews and model

This notebook creates a model for predicting health inspection violations using Yelp user review data. See earlier parts for exploratory data analysis involving the other data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('halverson')

### Load the training data:

In [2]:
df = pd.read_csv('data/training_labels.txt', parse_dates=['date'])
df.rename(columns={'date':'inspect_date'}, inplace=True)
df['weighted_violations'] = 1 * df['*'] + 3 * df['**'] + 5 * df['***']
df.head()

,id,inspect_date,restaurant_id,*,**,***,weighted_violations
0,589,2010-02-02,KAoKWjog,3,0,1,8
1,28589,2009-12-10,p038M4om,2,0,0,2
2,31170,2008-07-16,B1oXymOV,4,0,0,4
3,2600,2015-01-30,m0oWJl3G,1,0,3,16
4,1016,2012-03-19,rJoQwlEV,0,0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27088 entries, 0 to 27087
Data columns (total 7 columns):
id                     27088 non-null int64
inspect_date           27088 non-null datetime64[ns]
restaurant_id          27088 non-null object
*                      27088 non-null int64
**                     27088 non-null int64
***                    27088 non-null int64
weighted_violations    27088 non-null int64
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 1.4+ MB


### Clean the training data by removing duplicate inspections

In [4]:
from helper_methods import drop_duplicate_inspections
df = df.sort_values(['restaurant_id', 'inspect_date'])
df = drop_duplicate_inspections(df, threshold=60)
df.head()

,id,inspect_date,restaurant_id,*,**,***,weighted_violations
1801,28144,2007-09-21,0ZED0WED,3,1,0,6
551,24765,2008-03-26,0ZED0WED,3,1,0,6
5460,25193,2008-10-08,0ZED0WED,6,2,4,32
3641,12775,2009-03-03,0ZED0WED,3,0,0,3
18452,25850,2009-07-23,0ZED0WED,1,0,2,11


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20288 entries, 1801 to 5854
Data columns (total 7 columns):
id                     20288 non-null int64
inspect_date           20288 non-null datetime64[ns]
restaurant_id          20288 non-null object
*                      20288 non-null int64
**                     20288 non-null int64
***                    20288 non-null int64
weighted_violations    20288 non-null int64
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 1.2+ MB


In [6]:
for column in df:
    print df.shape[0], df[column].unique().size, column

20288 20288 id
20288 2058 inspect_date
20288 1851 restaurant_id
20288 44 *
20288 8 **
20288 18 ***
20288 102 weighted_violations


### Load the data to relate restaurant id to business id

In [7]:
trans = pd.read_csv('data/restaurant_ids_to_yelp_ids.csv')
trans = trans[trans['yelp_id_1'].isnull()]
trans.drop(['yelp_id_1', 'yelp_id_2', 'yelp_id_3'], axis=1, inplace=True)
trans.columns = ['restaurant_id', 'business_id']
trans.head()

,restaurant_id,business_id
0,Y1Em4GOw,5Kdf1DGbRScRk6Cx3jaX8w
1,KAoKP6Og,Urw6NASrebP6tyFdjwjkwQ
2,WeEe7eoa,xlOE7jqbW1Q_PrvLBVlegQ
3,V430mqoB,ktYpqtygWIJ2RjVPGTxNaA
4,ekE4Qz32,n8CsQy7Iy1IMhP85hPVKPA


In [8]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1826 entries, 0 to 1866
Data columns (total 2 columns):
restaurant_id    1826 non-null object
business_id      1826 non-null object
dtypes: object(2)
memory usage: 42.8+ KB


In [9]:
# uncomment the lines below to work with restaurants that have had multiple owners
# the problem with this is that it is not possible to determine when ownership
# changed

#from helper_methods import biz2yelp
#trans = biz2yelp()
#trans.columns = ['restaurant_id', 'business_id']
#trans.head()

In [10]:
df_trans = pd.merge(trans, df, on='restaurant_id', how='inner')
df_trans.head()

,restaurant_id,business_id,id,inspect_date,*,**,***,weighted_violations
0,Y1Em4GOw,5Kdf1DGbRScRk6Cx3jaX8w,17,2015-02-06,1,0,0,1
1,KAoKP6Og,Urw6NASrebP6tyFdjwjkwQ,9337,2007-06-27,0,0,0,0
2,KAoKP6Og,Urw6NASrebP6tyFdjwjkwQ,5052,2008-03-25,8,0,0,8
3,KAoKP6Og,Urw6NASrebP6tyFdjwjkwQ,12585,2009-04-23,5,0,2,15
4,KAoKP6Og,Urw6NASrebP6tyFdjwjkwQ,3583,2010-03-25,4,0,0,4


In [12]:
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19836 entries, 0 to 19835
Data columns (total 8 columns):
restaurant_id          19836 non-null object
business_id            19836 non-null object
id                     19836 non-null int64
inspect_date           19836 non-null datetime64[ns]
*                      19836 non-null int64
**                     19836 non-null int64
***                    19836 non-null int64
weighted_violations    19836 non-null int64
dtypes: datetime64[ns](1), int64(5), object(2)
memory usage: 1.4+ MB


In [13]:
np.random.seed(0)
msk = np.random.rand(df_trans.shape[0]) < 0.8
df_train = df_trans[msk]
df_test = df_trans[~msk]

### At this point we have our train-test split. Now we test the null model and the model where we predict the average value:

In [14]:
from sklearn.metrics import mean_squared_error as mse

y_true_train = df_train['*'].values
y_pred_train = 3 * np.ones(df_train.shape[0])

y_true_test = df_test['*'].values
y_pred_test = 3 * np.ones(df_test.shape[0])

print mse(y_true_train, y_pred_train)
print mse(y_true_test, y_pred_test)

18.2735112678
18.0913924051


Now try the avg_violations predictions (here it is necessary to compute the averages using the combined train and test data since we need at least one record for the test set predictions):

In [15]:
avg_violations = df_trans.groupby('restaurant_id').agg({'*': [np.size, np.mean, np.sum], '**': [np.mean, np.sum], '***': [np.mean, np.sum], 'weighted_violations': [np.mean, np.sum]})
avg_violations.head(5)

weighted_violations         *                     ***      \
                             mean  sum size      mean sum      mean sum   
restaurant_id                                                             
0ZED0WED                12.500000  200   16  4.312500  69  1.187500  19   
0ZED1B3D                 5.909091   65   11  2.363636  26  0.545455   6   
0ZED4ED9                 7.200000   36    5  4.200000  21  0.600000   3   
0ZED4pED                19.727273  217   11  7.000000  77  2.000000  22   
0ZED543D                 6.200000   31    5  3.800000  19  0.000000   0   

                     **      
                   mean sum  
restaurant_id                
0ZED0WED       0.750000  12  
0ZED1B3D       0.272727   3  
0ZED4ED9       0.000000   0  
0ZED4pED       0.909091  10  
0ZED543D       0.800000   4

In [16]:
avg_vio_train = pd.merge(avg_violations, df_train, left_index=True, right_on='restaurant_id', how='right')
y_pred_train = avg_vio_train[[('*', 'mean')]].values
y_true_train = avg_vio_train[['*']].values

avg_vio_test = pd.merge(avg_violations, df_test, left_index=True, right_on='restaurant_id', how='right')
y_pred_test = avg_vio_test[[('*', 'mean')]].values
y_true_test = avg_vio_test[['*']].values

print 'MSE (train) = %.1f' % mse(y_true_train, y_pred_train)
print 'MSE (test) = %.1f' % mse(y_true_test, y_pred_test)

MSE (train) = 13.4
MSE (test) = 13.3


### What if we only considered the violation history up to the inspection date?

In [17]:
def mean_violations_up_to_inspection_date(x, r, d):
    xf = x[(x['restaurant_id'] == r) & (x['inspect_date'] < d)]
    return xf.mean()['*'] if not xf.empty else 3.0

In [18]:
y_train = df_train.apply(lambda row: mean_violations_up_to_inspection_date(df_train, row['restaurant_id'], row['inspect_date']), axis=1)
y_pred_train = y_train.values
y_true_train = df_train['*'].values

y_test = df_test.apply(lambda row: mean_violations_up_to_inspection_date(df_test, row['restaurant_id'], row['inspect_date']), axis=1)
y_pred_test = y_test.values
y_true_test = df_test['*'].values

print 'MSE (train) = %.1f' % mse(y_true_train, y_pred_train)
print 'MSE (test) = %.1f' % mse(y_true_test, y_pred_test)

MSE (train) = 19.2
MSE (test) = 22.5


### Fit a linear model over the entire range

In [19]:
def linear_regression_all_inspections(df, r, d):
    xf = df[df['restaurant_id'] == r]
    if xf.empty or xf.shape[0] < 2: return 3.0
    x = (xf.inspect_date - pd.to_datetime('2006-10-04')) / np.timedelta64(1, 'D')
    x = x.values / (9 * 365.0)
    y = xf['*'].values
    y.dtype = np.float64
    a, b = np.polyfit(x, y, 1)
    x_ = (d - pd.to_datetime('2006-10-04')) / np.timedelta64(1, 'D')
    return a * x_ + b

In [20]:
y_train = df_train.apply(lambda row: linear_regression_all_inspections(df_train, row['restaurant_id'], row['inspect_date']), axis=1)
y_pred_train = y_train.values
y_true_train = df_train['*'].values

y_test = df_test.apply(lambda row: linear_regression_all_inspections(df_test, row['restaurant_id'], row['inspect_date']), axis=1)
y_pred_test = y_test.values
y_true_test = df_test['*'].values

print 'MSE (train) = %.1f' % mse(y_true_train, y_pred_train)
print 'MSE (test) = %.1f' % mse(y_true_test, y_pred_test)

MSE (train) = 31.5
MSE (test) = 30.4


### Fit a linear model up to the inspection date:

In [21]:
def linear_regression_previous_inspections(df, r, d):
    xf = df[(df['restaurant_id'] == r) & (df['inspect_date'] < d)]
    if xf.empty or xf.shape[0] < 2: return 3.0
    x = (xf.inspect_date - pd.to_datetime('2006-10-04')) / np.timedelta64(1, 'D')
    x = x.values / (9 * 365.0)
    y = xf['*'].values
    y.dtype = np.float64
    a, b = np.polyfit(x, y, 1)
    x_ = (d - pd.to_datetime('2006-10-04')) / np.timedelta64(1, 'D')
    return a * x_ + b

In [22]:
y_train = df_train.apply(lambda row: linear_regression_previous_inspections(df_train, row['restaurant_id'], row['inspect_date']), axis=1)
y_pred_train = y_train.values
y_true_train = df_train['*'].values

y_test = df_test.apply(lambda row: linear_regression_previous_inspections(df_test, row['restaurant_id'], row['inspect_date']), axis=1)
y_pred_test = y_test.values
y_true_test = df_test['*'].values

print 'MSE (train) = %.1f' % mse(y_true_train, y_pred_train)
print 'MSE (test) = %.1f' % mse(y_true_test, y_pred_test)

MSE (train) = 28.2
MSE (test) = 22.8


### With these simple models we move on to NLP and using the Yelp data

In [23]:
from helper_methods import read_json
df_rev = read_json('data/yelp_academic_dataset_review.json')
df_rev.rename(columns={'date':'review_date'}, inplace=True)
df_rev.drop(['type', 'user_id', 'votes'], axis=1, inplace=True)
df_rev.head(3)

,business_id,review_date,review_id,stars,text
0,Jp9svt7sRT4zwdbzQ8KQmw,2005-08-26,OeT5kgUOe3vcN7H6ImVmZQ,3,This is a pretty typical cafe. The sandwiches...
1,Jp9svt7sRT4zwdbzQ8KQmw,2005-11-23,qq3zF2dDUh3EjMDuKBqhEA,3,I agree with other reviewers - this is a prett...
2,Jp9svt7sRT4zwdbzQ8KQmw,2005-11-23,i3eQTINJXe3WUmyIpvhE9w,3,"Decent enough food, but very overpriced. Just ..."


### Exploratory data analysis of the reviews concludes here. Now we begin the model building:

Our strategy is to pair the inspection results with the reviews that were written in the t_days before the inspection. For instance, if the inspection was on March 1 then we would consider all reviews written between then and March 1 minus t_days. We do not consider reviews beyond the inspection date because the way the contest works we will be predicting violations using only past data.

In [24]:
df_train.head(3)

,restaurant_id,business_id,id,inspect_date,*,**,***,weighted_violations
0,Y1Em4GOw,5Kdf1DGbRScRk6Cx3jaX8w,17,2015-02-06,1,0,0,1
1,KAoKP6Og,Urw6NASrebP6tyFdjwjkwQ,9337,2007-06-27,0,0,0,0
2,KAoKP6Og,Urw6NASrebP6tyFdjwjkwQ,5052,2008-03-25,8,0,0,8


In [25]:
df_test.head(3)

,restaurant_id,business_id,id,inspect_date,*,**,***,weighted_violations
7,WeEe7eoa,xlOE7jqbW1Q_PrvLBVlegQ,28427,2012-12-30,0,0,0,0
8,V430mqoB,ktYpqtygWIJ2RjVPGTxNaA,28794,2007-08-03,4,0,2,14
13,ekE4Qz32,n8CsQy7Iy1IMhP85hPVKPA,10665,2011-04-21,3,0,1,8


In [26]:
df_rev[['business_id', 'text', 'review_date']].head(3)

,business_id,text,review_date
0,Jp9svt7sRT4zwdbzQ8KQmw,This is a pretty typical cafe. The sandwiches...,2005-08-26
1,Jp9svt7sRT4zwdbzQ8KQmw,I agree with other reviewers - this is a prett...,2005-11-23
2,Jp9svt7sRT4zwdbzQ8KQmw,"Decent enough food, but very overpriced. Just ...",2005-11-23


In [27]:
def join_and_filter(df_violations, df_reviews, t_window):
    """Join the violations and reviews. Filter reviews that are not with t_window days of the inspection date."""
    xl = pd.merge(df_violations, df_reviews, on='business_id', how='left')
    xl = xl[(xl['inspect_date'] >= xl['review_date']) & ((xl['inspect_date'] - xl['review_date']) / np.timedelta64(1, 'D') <= t_window)]
    xl.drop(['id', 'weighted_violations', 'business_id', 'review_id'], axis=1, inplace=True)
    return xl

In [28]:
def join_and_filter2(df_violations, df_reviews, t_window):
    """Join the violations and reviews. Filter reviews that are not with t_window days of the inspection date."""
    xl = pd.merge(df_violations, df_reviews, on='business_id', how='left')
    xl = xl[(xl['inspect_date'] >= xl['review_date']) & ((xl['inspect_date'] - xl['review_date']) / np.timedelta64(1, 'D') <= t_window)]
    xl.drop(['weighted_violations', 'business_id', 'review_id'], axis=1, inplace=True)
    xl.text = xl.text.apply(lambda x: x + ' ||| ')
    xl = xl.groupby('id').agg({'text':[np.sum], '*':['first'], '**':['first'], '***':['first']})
    xl.columns = ['text', '*', '**', '***']
    return xl

Create a routine to process the text:

In [29]:
import re
from nltk.stem.porter import PorterStemmer

def review_to_words(raw_review):
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review)
    return letters_only.lower().split()

def review_to_words_porter(raw_review):
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review)
    words = letters_only.lower().split()
    porter = PorterStemmer()
    return [porter.stem(word) for word in words]

### Prepare the train and test data

In [30]:
t_days = 60

xl = join_and_filter2(df_train, df_rev, t_days)
X_train = xl['text'].values
y_train = xl['*'].values

xl = join_and_filter2(df_test, df_rev, t_days)
X_test = xl['text'].values
y_true_test = xl['*'].values

Because we work with a sparse matrix, we will not standardize each column since that would lead to a dense matrix and high memory demands.

In [49]:
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
stops = stopwords.words("english")

tfidf = TfidfVectorizer(max_features=2000, ngram_range=(1, 3), smooth_idf=True, norm='l2')
param_grid = [{'vect__stop_words': [stops, None], 'vect__tokenizer': [review_to_words, review_to_words_porter]}]
#lr_tfidf = Pipeline([('vect', tfidf), ('linreg', Ridge(alpha=10.0))])
lr_tfidf = Pipeline([('vect', tfidf), ('linreg', RandomForestRegressor(n_estimators=20))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='mean_squared_error', cv=3, verbose=1, n_jobs=-1)
gs_lr_tfidf = gs_lr_tfidf.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 18.3min finished


In [50]:
print gs_lr_tfidf.best_params_.keys()
print gs_lr_tfidf.best_params_.values()

['vect__tokenizer', 'vect__stop_words']
[<function review_to_words_porter at 0x1705dca28>, None]


In [51]:
y_train_pred = gs_lr_tfidf.predict(X_train)
print 'Linear model: mse =', mse(y_train, y_train_pred)

Linear model: mse = 3.1651900197


### Test data

In [52]:
y_pred_test = gs_lr_tfidf.predict(X_test)
print 'Linear model: mse =', mse(y_true_test, y_pred_test)

Linear model: mse = 17.7556489883
